In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import util
# import fit_glm_helpers as fgh


In [3]:
dir_data = Path('/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2')
dir_output = Path('/Users/josh/Desktop/example_output_folder')

lst_dict_inputdata = [
    {'session_id': 'WT63_11082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT69_12192021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT69_12192021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},
]

dir_output.mkdir(parents=True, exist_ok=True)

In [4]:
df_signal = pd.read_csv(lst_dict_inputdata[0]['filepath_signal'])
df_trial = pd.read_csv(lst_dict_inputdata[0]['filepath_trial'])

In [5]:
df_signal

,Ch1,Ch2,Ch5,Ch6,centerOcc,centerIn,centerOut,rightOcc,rightIn,rightOut,rightLick,leftOcc,leftIn,leftOut,leftLick,reward,noreward
0,-2.515587,0,0.087224,0,0,0,0,0,0,0,0,1,0,0,1,0,1
1,-1.660977,0,-0.159638,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,-2.155949,0,-0.408390,0,0,0,0,0,0,0,0,1,0,0,1,0,1
3,-0.868106,0,-0.799495,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,-0.477200,0,-0.422909,0,0,0,0,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29845,-2.426426,0,0.590765,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29846,-2.861054,0,-1.745552,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29847,-2.314100,0,-0.742046,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29848,-1.455169,0,-0.879324,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df_trial

,hasAllPhotometryData,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex,photometryFirstLickIndex,choseLeft,choseRight,leftRewardProb,rightRewardProb,wasRewarded,word
0,0,0,0,0,0,0,0,1,0.85,0.15,0,NaN
1,0,0,0,0,0,0,0,1,0.85,0.15,0,aa
2,0,0,0,0,0,0,1,0,0.85,0.15,1,aB
3,0,0,0,0,0,0,0,1,0.85,0.15,0,Ab
4,0,0,0,0,0,0,0,1,0.85,0.15,0,aa
...,...,...,...,...,...,...,...,...,...,...,...,...
341,1,24370,24371,24389,24413,24391,1,0,0.85,0.15,1,aB
342,1,25218,25225,25248,25253,0,0,1,0.85,0.15,0,Ab
343,1,25765,25774,25826,25831,0,1,0,0.85,0.15,0,ab
344,1,29025,29030,29065,29071,0,0,1,0.85,0.15,0,ab


In [7]:
columnName_alignment_trial_start = 'photometryCenterInIndex'
columnName_alignment_trial_end = 'photometrySideOutIndex'

# Note: Alignment values of 0 for Matlab-indexed trial tables will be treated as "no-data" values
# and and -1 for Python-indexed trial tables. Matlab-indexed trial tables should only have values
# >= 0 in and >= -1 in Python.
trial_index_columns = [
    'photometryCenterInIndex',
    'photometryCenterOutIndex',
    'photometrySideInIndex',
    'photometrySideOutIndex',
]

trial_num_columns = [
    'leftRewardProb',
    'rightRewardProb',
]

trial_cat_columns = [
    'hasAllPhotometryData',
    'wasRewarded',
    'word',
]

signal_dense_columns = [
    'Ch1',
    'Ch5',
]

signal_sparse_columns = [
    'centerOcc',
    'centerIn',
    'centerOut',
    'rightOcc',
    'rightIn',
    'rightOut',
    'rightLick',
    'leftOcc',
    'leftIn',
    'leftOut',
    'leftLick',
]

In [8]:
df_trial = df_trial[(df_trial[trial_index_columns] == 0).sum(axis=1) == 0]
df_trial_indices = df_trial[trial_index_columns]
df_trial_num = df_trial[trial_num_columns]
df_trial_cat = df_trial[trial_cat_columns]

df_signal_dense = df_signal[signal_dense_columns]
df_signal_toSparse = df_signal[signal_sparse_columns]

In [9]:
display(df_trial.head())
display(df_signal.head())
display(df_trial_indices.head())
display(df_trial_num.head())
display(df_trial_cat.head())
display(df_signal_dense.head())
display(df_signal_toSparse.head())

,hasAllPhotometryData,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex,photometryFirstLickIndex,choseLeft,choseRight,leftRewardProb,rightRewardProb,wasRewarded,word
19,1,1223,1226,1240,1266,1243,1,0,0.85,0.15,1,aA
20,1,1291,1292,1307,1335,1310,1,0,0.85,0.15,1,AA
21,1,1451,1453,1468,1494,1470,1,0,0.85,0.15,1,AA
22,1,1508,1510,1524,1553,1526,1,0,0.85,0.15,1,AA
23,1,1566,1567,1581,1614,1585,1,0,0.85,0.15,1,AA


,Ch1,Ch2,Ch5,Ch6,centerOcc,centerIn,centerOut,rightOcc,rightIn,rightOut,rightLick,leftOcc,leftIn,leftOut,leftLick,reward,noreward
0,-2.515587,0,0.087224,0,0,0,0,0,0,0,0,1,0,0,1,0,1
1,-1.660977,0,-0.159638,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,-2.155949,0,-0.408390,0,0,0,0,0,0,0,0,1,0,0,1,0,1
3,-0.868106,0,-0.799495,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,-0.477200,0,-0.422909,0,0,0,0,0,0,0,0,1,0,0,1,0,1


,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex
19,1223,1226,1240,1266
20,1291,1292,1307,1335
21,1451,1453,1468,1494
22,1508,1510,1524,1553
23,1566,1567,1581,1614


,leftRewardProb,rightRewardProb
19,0.85,0.15
20,0.85,0.15
21,0.85,0.15
22,0.85,0.15
23,0.85,0.15


,hasAllPhotometryData,wasRewarded,word
19,1,1,aA
20,1,1,AA
21,1,1,AA
22,1,1,AA
23,1,1,AA


,Ch1,Ch5
0,-2.515587,0.087224
1,-1.660977,-0.159638
2,-2.155949,-0.408390
3,-0.868106,-0.799495
4,-0.477200,-0.422909


,centerOcc,centerIn,centerOut,rightOcc,rightIn,rightOut,rightLick,leftOcc,leftIn,leftOut,leftLick
0,0,0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,1,0,0,1


In [10]:
df_trial_cat

,hasAllPhotometryData,wasRewarded,word
19,1,1,aA
20,1,1,AA
21,1,1,AA
22,1,1,AA
23,1,1,AA
...,...,...,...
341,1,1,aB
342,1,0,Ab
343,1,0,ab
344,1,0,ab


In [11]:
nrows = df_signal.shape[0]

matrix_signal_sparse = util.Matrix.from_pd_dense_num(df_signal_toSparse)
matrix_signal_dense = util.Matrix.from_pd_dense_num(df_signal_dense)

matrix_trial_indices = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1)
matrix_trial_num = util.Matrix.from_pd_sparse_num(nrows, values=df_trial_num)
matrix_trial_in = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=df_trial_num, sparse_value=np.nan)

matrix_trial_cat = util.Matrix.from_pd_sparse_num(nrows, values=df_trial_cat, sparse_value=np.nan)
matrix_trial_ic = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=df_trial_cat, sparse_value=np.nan)

assert np.all(df_signal_toSparse == matrix_signal_sparse.to_pd()), "df_signal_toSparse is failed to convert to sparse"
assert np.all(df_signal_dense == matrix_signal_dense.to_pd()), "df_signal_dense is failed to convert to sparse"


In [12]:
df_signal_toSparse

,centerOcc,centerIn,centerOut,rightOcc,rightIn,rightOut,rightLick,leftOcc,leftIn,leftOut,leftLick
0,0,0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
29845,0,0,0,0,0,0,0,0,0,0,0
29846,0,0,0,0,0,0,0,0,0,0,0
29847,0,0,0,0,0,0,0,0,0,0,0
29848,0,0,0,0,0,0,0,0,0,0,0


In [13]:
matrix_signal_sparse.to_pd()

,centerOcc,centerIn,centerOut,rightOcc,rightIn,rightOut,rightLick,leftOcc,leftIn,leftOut,leftLick
0,0,0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
29845,0,0,0,0,0,0,0,0,0,0,0
29846,0,0,0,0,0,0,0,0,0,0,0
29847,0,0,0,0,0,0,0,0,0,0,0
29848,0,0,0,0,0,0,0,0,0,0,0


In [14]:
mt = util.Matrix([matrix_trial_in, matrix_trial_indices])
mtpd = mt.to_pd()
with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
    display(mtpd[((~mtpd[trial_num_columns].isna()).sum(axis=1) > 0)&(((mtpd[list(zip(['']*len(trial_index_columns), trial_index_columns))] != 0)).sum(axis=1) > 0)])

leftRewardProb         rightRewardProb  \
      photometryCenterInIndex photometryCenterInIndex   
1222                     0.85                    0.15   
1225                      NaN                     NaN   
1239                      NaN                     NaN   
1265                      NaN                     NaN   
1290                     0.85                    0.15   
...                       ...                     ...   
29070                     NaN                     NaN   
29104                    0.85                    0.15   
29107                     NaN                     NaN   
29128                     NaN                     NaN   
29161                     NaN                     NaN   

                leftRewardProb          rightRewardProb        leftRewardProb  \
      photometryCenterOutIndex photometryCenterOutIndex photometrySideInIndex   
1222                       NaN                      NaN                   NaN   
1225                      0.85                     0.15                   NaN   
1239                       NaN                      NaN                  0.85   
1265                       NaN                      NaN                   NaN   
1290                       NaN                      NaN                   NaN   
...                        ...                      ...                   ...   
29070                      NaN                      NaN                   NaN   
29104                      NaN                      NaN                   NaN   
29107                     0.85                     0.15                   NaN   
29128                      NaN                      NaN                  0.85   
29161                      NaN                      NaN                   NaN   

            rightRewardProb         leftRewardProb        rightRewardProb  \
      photometrySideInIndex photometrySideOutIndex photometrySideOutIndex   
1222                    NaN                    NaN                    NaN   
1225                    NaN                    NaN                    NaN   
1239                   0.15                    NaN                    NaN   
1265                    NaN                   0.85                   0.15   
1290                    NaN                    NaN                    NaN   
...                     ...                    ...                    ...   
29070                   NaN                   0.85                   0.15   
29104                   NaN                    NaN                    NaN   
29107                   NaN                    NaN                    NaN   
29128                  0.15                    NaN                    NaN   
29161                   NaN                   0.85                   0.15   

                                                                              \
      photometryCenterInIndex photometryCenterOutIndex photometrySideInIndex   
1222                      1.0                      0.0                   0.0   
1225                      0.0                      1.0                   0.0   
1239                      0.0                      0.0                   1.0   
1265                      0.0                      0.0                   0.0   
1290                      1.0                      0.0                   0.0   
...                       ...                      ...                   ...   
29070                     0.0                      0.0                   0.0   
29104                     1.0                      0.0                   0.0   
29107                     0.0                      1.0                   0.0   
29128                     0.0                      0.0                   1.0   
29161                     0.0                      0.0                   0.0   

                              
      photometrySideOutIndex  
1222                     0.0  
1225                     0.0  
1239                     0.0  
1265                 

In [15]:
mt.shift([-2, -1, 0, 1, 2]).to_pd().sort_index(axis=1)

\
      photometryCenterInIndex                     photometryCenterOutIndex   
                           -2   -1    0    1    2                       -2   
0                         0.0  0.0  0.0  0.0  0.0                      0.0   
1                         0.0  0.0  0.0  0.0  0.0                      0.0   
2                         0.0  0.0  0.0  0.0  0.0                      0.0   
3                         0.0  0.0  0.0  0.0  0.0                      0.0   
4                         0.0  0.0  0.0  0.0  0.0                      0.0   
...                       ...  ...  ...  ...  ...                      ...   
29845                     0.0  0.0  0.0  0.0  0.0                      0.0   
29846                     0.0  0.0  0.0  0.0  0.0                      0.0   
29847                     0.0  0.0  0.0  0.0  0.0                      0.0   
29848                     0.0  0.0  0.0  0.0  0.0                      0.0   
29849                     0.0  0.0  0.0  0.0  0.0                      0.0   

                           ...       rightRewardProb                  \
                           ... photometrySideInIndex                   
        -1    0    1    2  ...                    -2  -1   0   1   2   
0      0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
1      0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
2      0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
3      0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
4      0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
...    ...  ...  ...  ...  ...                   ...  ..  ..  ..  ..   
29845  0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
29846  0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
29847  0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
29848  0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   
29849  0.0  0.0  0.0  0.0  ...                   NaN NaN NaN NaN NaN   

                                              
      photometrySideOutIndex                  
                          -2  -1   0   1   2  
0                        NaN NaN NaN NaN NaN  
1                        NaN NaN NaN NaN NaN  
2                        NaN NaN NaN NaN NaN  
3                        NaN NaN NaN NaN NaN  
4                        NaN NaN NaN NaN NaN  
...                      ...  ..  ..  ..  ..  
29845                    NaN NaN NaN NaN NaN  
29846                    NaN NaN NaN NaN NaN  
29847                    NaN NaN NaN NaN NaN  
29848                    NaN NaN NaN NaN NaN  
29849                    NaN NaN NaN NaN NaN  

[29850 rows x 60 columns]

In [16]:
mt

Matrix(vectors={('leftRewardProb', ('photometryCenterInIndex',)): VectorSparse(name=leftRewardProb, nrows=29850, values=[0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85
 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85
 0.85 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15
 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15
 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15
 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85
 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85
 0.85 0.85 0.85 0.85 0.85 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15
 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15
 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.85
 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85
 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85
 0.85 0.85 0.85 0.85 0.85 0.

In [34]:
lrp = mt[(None, 'photometryCenterInIndex')].to_pd()
# lrp[(~lrp.isna()).sum(axis=1) > 0]
lrp[((~lrp.isna()).sum(axis=1) > 0)&(lrp == 1).sum(axis=1) > 0]

,leftRewardProb,rightRewardProb,
,photometryCenterInIndex,photometryCenterInIndex,photometryCenterInIndex
1222,0.85,0.15,1.0
1290,0.85,0.15,1.0
1450,0.85,0.15,1.0
1507,0.85,0.15,1.0
1565,0.85,0.15,1.0
...,...,...,...
24369,0.85,0.15,1.0
25217,0.85,0.15,1.0
25764,0.85,0.15,1.0


In [33]:
m = util.Matrix([mt[(None, 'photometryCenterInIndex')], mt[(None, 'photometryCenterInIndex')].shift([-1, 1])]).to_pd()
m[(m['']['photometryCenterInIndex'] > 0).sum(axis=1) > 0]

,leftRewardProb,rightRewardProb,,leftRewardProb,rightRewardProb,,leftRewardProb,rightRewardProb,
,photometryCenterInIndex,photometryCenterInIndex,photometryCenterInIndex,photometryCenterInIndex,photometryCenterInIndex,photometryCenterInIndex,photometryCenterInIndex,photometryCenterInIndex,photometryCenterInIndex
,0,0,0,-1,-1,-1,1,1,1
1221,NaN,NaN,0.0,0.85,0.15,1.0,NaN,NaN,0.0
1222,0.85,0.15,1.0,NaN,NaN,0.0,NaN,NaN,0.0
1223,NaN,NaN,0.0,NaN,NaN,0.0,0.85,0.15,1.0
1289,NaN,NaN,0.0,0.85,0.15,1.0,NaN,NaN,0.0
1290,0.85,0.15,1.0,NaN,NaN,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...
29024,0.85,0.15,1.0,NaN,NaN,0.0,NaN,NaN,0.0
29025,NaN,NaN,0.0,NaN,NaN,0.0,0.85,0.15,1.0


In [52]:
mt.shift(range(-30, 31)).to_pd()['']

,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex,photometryCenterInIndex,photometryCenterOutIndex,...,photometrySideInIndex,photometrySideOutIndex,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex
,-30,-30,-30,-30,-29,-29,-29,-29,-28,-28,...,28,28,29,29,29,29,30,30,30,30
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
